# Tic-Tac-Toe and Connect-4 using Mini-Max

In [11]:
class TicTacToe:
    """ Tic-Tac Toe Board 
        
        Board is represented through a board object
        
        1 ¦ 2 ¦ 3
        --+---+--
        4 ¦ 5 ¦ 6
        --+---+--
        7 ¦ 8 ¦ 9
    """
    def __init__(self):
        self.spaces = 9
        self.blank = " "
        self.legal_players = ["X","O"]
        self.base =      " {} ¦ {} ¦ {} \n" + \
                          "---+---+--- \n" + \
                         " {} ¦ {} ¦ {} \n" + \
                          "---+---+--- \n" + \
                         " {} ¦ {} ¦ {} \n" 
        self.winning = [[0,1,2], # top horizontal
                        [3,4,5], # middle horizontal
                        [6,7,8], # bottom horizontal
                        [0,3,6], # left vertical
                        [1,4,7], # middle vertical
                        [2,5,8], # right vertical
                        [0,4,8], # diagonal \
                        [2,4,6]  # diagonal /
                       ]
        self.reset()
    
    def reset(self):
        """ Resets the board """
        self.board = [" "] * self.spaces

    def __str__(self):
        """ String representation of a game """
        return self.base.format(*self.board)
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.board == other.board
        return False

    def __hash__(self):
        return hash("".join(self.board))
    
    def copy(self):
        """ Creates a copy of the game object """
        new_board = self.__class__()
        new_board.board = self.board[:]
        return new_board
        
    def get_moves(self):
        """ Returns open moves on the board """
        return [idx + 1 for idx, player in enumerate(self.board) if player == self.blank]
    
    def move(self, player, move):
        """ Updates the board based on player and move 
            
            example: 
                game.move("X", 4)                
        """
        player = player.upper()
        assert move in self.get_moves(), "Illegal move"
        assert player in self.legal_players, "Illegal player"
        self.board[move - 1] = player
        

    def gameover(self):
        """ Returns whether there are any legal moves left """
        return len(self.get_moves()) == 0 or self.winner() is not None
    
    def winner(self):
        """ Returns the winner of a game, 'Draw' or None if game is ongoing """
        for winner in self.winning:
            players = [self.board[pos] for pos in winner]
            s_players = set(players)
            if len(s_players) == 1 and self.blank not in s_players:
                return players[0]
        
        if self.blank not in set(self.board):
            return "Draw"

    def score_game(self, player):
        """ Scores a game based relative to player provided """
        winner = self.winner()
        player = player.upper()
        alt_players = [alt_player for alt_player in self.legal_players if alt_player != player]

        if winner == player:
            return 1
        elif winner in alt_players:
            return -1
        return 0

In [81]:
def play(game, bot):
    human = None
    while human is None:
        human = input("Select player: {} ".format(game.legal_players)).upper()
        if human not in game.legal_players:
            print("Invalid option")
            human = None
            
    comp = [alt_player for alt_player in game.legal_players if alt_player != human][0]
    turn = game.legal_players[0]
    while not game.gameover():    
        if comp == turn:
            move = bot.play(game, comp)
        else:
            print(game)
            move = None
            while move is None:
                move = input("Input move: ")
                if move.upper()[:1] == "Q":
                    print("Quitter...")
                    return
                elif move.upper()[:1] == "H":
                    print(game.__doc__)
                    continue
                if move.isdigit():
                    move = int(move)
        if move in game.get_moves():
            game.move(turn, move)
            turn = comp if turn == human else human
        else:
            print("Illegal move {}. Q to quit. H for help".format(move))
    print(game)
    if game.winner() == human:
        print("You won!")
    elif game.winner() == comp:
        print("You lost :-(")
    else:
        print("It's a draw")
    

In [3]:
class MiniMaxBot:
    def __init__(self):
        """ Returns best move given game and player """
        self.memo = {}
        
    def play(self, game, player):
        return self._mini_max(game, player)[0]

    def _mini_max(self, game, player):
        """ Helper function for get_best_move. Returns best move and score given game and player """
        if player not in self.memo:
            self.memo[player] = {}
            
        player_memo = self.memo[player]
        
        if game not in player_memo:
            # Check to see if we've already seen this state
            if game.gameover():
                # Game over, no moves possible
                best_move = None
                best_score = game.score_game(player)
            else:
                alt_player = [alt_player for alt_player in game.legal_players if alt_player != player][0]
                moves = game.get_moves() # Get available moves
                best_score = float("-inf") # Default to worst possible score to ensure any move is selected

                for move in moves:
                    clone = game.copy() # Make a copy of the game to run recursively
                    clone.move(player, move) # Make the proposed move
                     # Figure out what the opponents best move would be (MINI)
                    _, score = self._mini_max(clone, player=alt_player)
                    score *= -1 # Since the game is zero-sum, what's bad for the opponent is good for us
                    if score > best_score: # Best our prior best score
                        best_move = move # Save the move
                        best_score = score # Update the best score
            self.memo[player][game] = (best_move, best_score) # Update the best move and score given the game
        return self.memo[player][game] # Return best move given player and game   


In [12]:
game = TicTacToe()

In [17]:
game.reset()
play(game, MiniMaxBot())

Select player: ['X', 'O'] x
   ¦   ¦   
---+---+--- 
   ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 5
 O ¦   ¦   
---+---+--- 
   ¦ X ¦   
---+---+--- 
   ¦   ¦   

Input move: 2
 O ¦ X ¦   
---+---+--- 
   ¦ X ¦   
---+---+--- 
   ¦ O ¦   

Input move: 6
 O ¦ X ¦   
---+---+--- 
 O ¦ X ¦ X 
---+---+--- 
   ¦ O ¦   

Input move: 7
 O ¦ X ¦ O 
---+---+--- 
 O ¦ X ¦ X 
---+---+--- 
 X ¦ O ¦   

Input move: 9
 O ¦ X ¦ O 
---+---+--- 
 O ¦ X ¦ X 
---+---+--- 
 X ¦ O ¦ X 

It's a draw


In [6]:
game.reset()
play(game, MiniMaxBot())

Select player: ['X', 'O'] o
 X ¦   ¦   
---+---+--- 
   ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 2
 X ¦ O ¦   
---+---+--- 
 X ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 3
 X ¦ O ¦ O 
---+---+--- 
 X ¦ X ¦   
---+---+--- 
   ¦   ¦   

Input move: 6
 X ¦ O ¦ O 
---+---+--- 
 X ¦ X ¦ O 
---+---+--- 
 X ¦   ¦   

You lost :-(


In [7]:
class MiniMaxTimeBot:
    def __init__(self, time_penalty = -0.01):
        """ Returns best move given game and player """
        self.time_penalty = time_penalty
        self.memo = {}
        
    def play(self, game, player):
        return self._mini_max(game, player)[0]

    def _mini_max(self, game, player, num_moves=0):
        """ Helper function for get_best_move. Returns best move and score given game and player """
        if player not in self.memo:
            self.memo[player] = {}
            
        player_memo = self.memo[player]
        
        if game not in player_memo:
            if game.gameover():
                best_move = None
                best_score = game.score_game(player)
            else:
                alt_player = [alt_player for alt_player in game.legal_players if alt_player != player][0]
                moves = game.get_moves()
                best_score = float("-inf")

                for move in moves:
                    clone = game.copy()
                    clone.move(player, move)
                    _, score = self._mini_max(clone, player=alt_player, num_moves=num_moves+1)
                    score *= -1 
                    score += self.time_penalty * num_moves # Add a time penalty for every move taken
                    if score > best_score:
                        best_move = move 
                        best_score = score
            self.memo[player][game] = (best_move, best_score) 
        return self.memo[player][game] 

In [13]:
game.reset()
play(game, MiniMaxTimeBot())

Select player: ['X', 'O'] o
 X ¦   ¦   
---+---+--- 
   ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 2
 X ¦ O ¦   
---+---+--- 
 X ¦   ¦   
---+---+--- 
   ¦   ¦   

Input move: 3
 X ¦ O ¦ O 
---+---+--- 
 X ¦   ¦   
---+---+--- 
 X ¦   ¦   

You lost :-(


In [14]:
class MiniMaxTimeAverageBot:
    def __init__(self, time_penalty = -0.01, sub_optimal_weight=0.1):
        """ Returns best move given game and player """
        self.time_penalty = time_penalty
        self.memo = {}
        self.sub_optimal_weight = sub_optimal_weight
        
    def play(self, game, player):
        return self._mini_max(game, player)[0]

    def _mini_max(self, game, player, num_moves=0):
        """ Helper function for get_best_move. Returns best move and score given game and player """
        if player not in self.memo:
            self.memo[player] = {}
            
        player_memo = self.memo[player]
        
        if game not in player_memo:
            if game.gameover():
                best_move = None
                best_score = game.score_game(player)
            else:
                alt_player = [alt_player for alt_player in game.legal_players if alt_player != player][0]
                moves = game.get_moves()
                best_score = float("-inf")
                sub_optimal_sum = 0
                for move in moves:
                    clone = game.copy()
                    clone.move(player, move)
                    _, score = self._mini_max(clone, player=alt_player, num_moves=num_moves+1)
                    score *= -1
                    score += self.time_penalty * num_moves
                    sub_optimal_sum += score

                    if score > best_score:
                        best_move = move 
                        best_score = score
                sub_optimal_sum -= best_score # Remove the best score from the sub-optimal running sum
                sub_optimal = sub_optimal_sum / len(moves) # Average the sub-optimal returns
                # Take a weighted average of the perfect and sub-optimal returns based on sub-optimal weight provided
                best_score = ((1 - self.sub_optimal_weight) * best_score) + (self.sub_optimal_weight * sub_optimal)
            self.memo[player][game] = (best_move, best_score)
        return self.memo[player][game] 

In [15]:
game.reset()
play(game,  MiniMaxTimeAverageBot(time_penalty=0,sub_optimal_weight=0.1))

Select player: ['X', 'O'] o
   ¦   ¦   
---+---+--- 
   ¦ X ¦   
---+---+--- 
   ¦   ¦   

Input move: 1
 O ¦ X ¦   
---+---+--- 
   ¦ X ¦   
---+---+--- 
   ¦   ¦   

Input move: 3
 O ¦ X ¦ O 
---+---+--- 
   ¦ X ¦   
---+---+--- 
   ¦ X ¦   

You lost :-(


In [63]:
class Connect:
    def __init__(self, connect=4, width=7, height=6):
        self.height = height
        self.width = width
        self.connect = connect
        self.legal_players = ["X", "O"]
        self.reset()
        
    def reset(self):
        self.board = [[" " for _ in range(self.width)] for _ in range(self.height)]
        
    def __str__(self):
        """ String representation of a game """
        board = ""
        for r in range(self.height):
            board += "|"
            for c in range(self.width):
                board += "{}|".format(self.board[r][c])
            board += "\n"
        return board[:-1]
        
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.board == other.board
        return False

    def __hash__(self):
        return hash("".join(sum(self.board,[])))
    
    def copy(self):
        """ Creates a copy of the game object """
        new_board = self.__class__(connect=self.connect, width=self.width, height=self.height)
        new_board.board = [x[:] for x in self.board]
        return new_board
    
    def get_moves(self):
        # Check if top row filled
        return [idx for idx, val in enumerate(self.board[0]) if val == " "]
    
    def _is_match(self, match):
        s_match = set(match)
        if len(s_match) == 1 and " " not in s_match:
            return match[0]
    
    def move(self, player, c):
        player = player.upper()
        assert c in self.get_moves(), "Illegal move"
        assert player in self.legal_players, "Illegal player"
        move_row = 0
        for r in reversed(range(self.height)):
            if self.board[r][c] == " ":
                move_row = r
                break
        self.board[move_row][c] = player
        
    def winner(self):
        # check horizontal
        for r in range(self.height):
            for c in range(self.width - self.connect + 1):
                match = [self.board[r][c+off] for off in range(self.connect)]
                if self._is_match(match) is not None:
                    return match[0]
                
        # check vertical
        for r in range(self.height - self.connect + 1):
            for c in range(self.width):
                match = [self.board[r+off][c] for off in range(self.connect)]
                if self._is_match(match) is not None:
                    return match[0]

        # check diagonal 
        for r in range(self.height - self.connect // 2 - 1):
            for c in range(self.width - self.connect // 2 - 1):
                # diagonal \
                match = [self.board[r + off][c + off] for off in range(self.connect)]
                if self._is_match(match) is not None:
                    return match[0]
                # diagonal /
                match = [self.board[off][self.width - off - 1] for off in range(self.connect)]
                if self._is_match(match) is not None:
                    return match[0]
                
    def score_game(self, player):
        if self.winner() == player:
            return 1
        elif self.winner() in self.legal_players:
            return -1
        return 0
    
    def gameover(self):
        return len(self.get_moves()) == 0 or self.winner() is not None


In [85]:
class MiniMaxTimeAverageLimitBot:
    def __init__(self, time_penalty = -0.01, sub_optimal_weight=0.1, limit=5):
        """ Returns best move given game and player """
        self.time_penalty = time_penalty
        self.memo = {}
        self.sub_optimal_weight = sub_optimal_weight
        self.limit = limit
        
    def play(self, game, player):
        return self._mini_max(game, player)[0]

    def _mini_max(self, game, player, num_moves=0):
        """ Helper function for get_best_move. Returns best move and score given game and player """
        if player not in self.memo:
            self.memo[player] = {}
            
        player_memo = self.memo[player]
        
        if game not in player_memo:
            if game.gameover():
                best_move = None
                best_score = game.score_game(player)
            elif num_moves > self.limit:
                best_move = game.get_moves()[0]
                best_score = 0
            else:
                alt_player = [alt_player for alt_player in game.legal_players if alt_player != player][0]
                moves = game.get_moves()
                best_score = float("-inf")
                sub_optimal_sum = 0
                for move in moves:
                    clone = game.copy()
                    clone.move(player, move)
                    _, score = self._mini_max(clone, player=alt_player, num_moves=num_moves+1)
                    score *= -1
                    score += self.time_penalty * num_moves
                    sub_optimal_sum += score

                    if score > best_score:
                        best_move = move 
                        best_score = score
                sub_optimal_sum -= best_score # Remove the best score from the sub-optimal running sum
                sub_optimal = sub_optimal_sum / len(moves) # Average the sub-optimal returns
                # Takea weighted average of the perfect and sub-optimal returns based on sub-optimal weight provided
                best_score = ((1 - self.sub_optimal_weight) * best_score) + (self.sub_optimal_weight * sub_optimal)
            self.memo[player][game] = (best_move, best_score)
        return self.memo[player][game] 

In [86]:
game = Connect(connect=4, height=6, width=7)

In [88]:
game.reset()
play(game,MiniMaxTimeAverageLimitBot())

Select player: ['X', 'O'] o
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|X| | | | | | |
Input move: 1
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|X| | | | | | |
|X|O| | | | | |
Input move: 0
| | | | | | | |
| | | | | | | |
|X| | | | | | |
|O| | | | | | |
|X| | | | | | |
|X|O| | | | | |
Input move: 1
| | | | | | | |
|X| | | | | | |
|X| | | | | | |
|O| | | | | | |
|X|O| | | | | |
|X|O| | | | | |
Input move: 0
|O| | | | | | |
|X| | | | | | |
|X| | | | | | |
|O|X| | | | | |
|X|O| | | | | |
|X|O| | | | | |
Input move: 3
|O| | | | | | |
|X| | | | | | |
|X| | | | | | |
|O|X| | | | | |
|X|O| | | | | |
|X|O|X|O| | | |
Input move: 3
|O| | | | | | |
|X| | | | | | |
|X|X| | | | | |
|O|X| | | | | |
|X|O| |O| | | |
|X|O|X|O| | | |
Input move: 1
|O|X| | | | | |
|X|O| | | | | |
|X|X| | | | | |
|O|X| | | | | |
|X|O| |O| | | |
|X|O|X|O| | | |
Input move: 4
|O|X| | | | | |
|X|O| | | | | |
|X|X| | | | | |
|O|X| | | | | |
|X|O|X|O| | | |
|X|O|X|O|O| 